#**load data**

In [1]:
import io
import os
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive/')

data_path='/content/gdrive/MyDrive/DL_project/dataset/'

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LeakyReLU
from tensorflow.keras import backend
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv(data_path+'WIX_call-options-black-scholes.csv')
call_df_21 = df.drop(columns=['Unnamed: 0','Unnamed: 0.1','UnderlyingSymbol','Exchange','OptionSymbol','Blank','Expiration',
                     'DataDate','Last','Volume','OpenInterest','IV','Delta','Gamma','Theta','Vega','Alias',
                     'sigma_5','sigma_10','sigma_30','black_scholes_pred_21'])
call_df_21=call_df_21.dropna()

In [4]:
call_df_21.head()

,UnderlyingPrice,Strike,Bid,Ask,sigma_21,date_diff,treasury_rate
1659,61.05,30.0,30.3,31.8,0.029875,16,1.43
1660,61.05,35.0,25.3,26.9,0.029875,16,1.43
1661,61.05,40.0,20.4,22.0,0.029875,16,1.43
1662,61.05,45.0,15.3,16.7,0.029875,16,1.43
1663,61.05,50.0,10.8,12.2,0.029875,16,1.43


In [5]:
call_df_21['UnderlyingPrice'] = call_df_21['UnderlyingPrice'].astype('float64') 
call_df_21['Strike'] = call_df_21['Strike'].astype('float64') 
call_df_21['Bid'] = call_df_21['Bid'].astype('float64') 
call_df_21['Ask'] = call_df_21['Ask'].astype('float64') 
call_df_21['sigma_21'] = call_df_21['sigma_21'].astype('float64') 
call_df_21['date_diff'] = call_df_21['date_diff'].astype('float64') 
call_df_21['treasury_rate'] = call_df_21['treasury_rate'].astype('float64') 


In [6]:
#call_df_21= call_df_21.apply(pd.to_numeric, errors='coerce')


In [7]:
call_df_21.dtypes

UnderlyingPrice    float64
Strike             float64
Bid                float64
Ask                float64
sigma_21           float64
date_diff          float64
treasury_rate      float64
dtype: object

In [8]:
call_df_21.to_csv('/content/gdrive/MyDrive/DL_project/dataset/call_df_21.csv')

#**TFX installation and pipeline**

In [9]:
!pip install tfx==0.23.1 tensorflow-data-validation==0.23.1

  Using cached https://files.pythonhosted.org/packages/91/ab/8a7b37278fb59f3688af01cd069a5a4d2f3383eea2a2f78ddea4c7be047a/grpcio-1.34.0-cp36-cp36m-manylinux2014_x86_64.whl
ERROR: google-cloud-spanner 1.19.1 has requirement google-cloud-core<2.0dev,>=1.4.1, but you'll have google-cloud-core 1.0.3 which is incompatible.
ERROR: google-cloud-bigtable 1.6.1 has requirement google-cloud-core<2.0dev,>=1.4.1, but you'll have google-cloud-core 1.0.3 which is incompatible.
  Found existing installation: grpcio 1.26.0
    Uninstalling grpcio-1.26.0:
      Successfully uninstalled grpcio-1.26.0


In [10]:

import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
#import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from tfx.utils.dsl_utils import external_input
from tfx.components import ImportExampleGen
from tfx.proto import example_gen_pb2

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

/usr/local/lib/python3.6/dist-packages/tfx/orchestration/data_types.py:191: UserWarning: RuntimeParameter is only supported on KubeflowDagRunner currently.
  warnings.warn('RuntimeParameter is only supported on KubeflowDagRunner '


In [11]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.3.0
TFX version: 0.23.1


In [12]:
import shutil
# This is the root directory for your TFX pip package installation.
_tfx_root = tfx.__path__[0]

# This is the directory containing the.
_taxi_root = os.path.join(_tfx_root, 'examples/options_pipeline')

# This is the path where your model will be pushed for serving.
_serving_model_dir = os.path.join(
    tempfile.mkdtemp(), 'serving_model/options_simple')

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

In [13]:
_data_root = tempfile.mkdtemp(prefix='tfx-data')

train_="/content/gdrive/MyDrive/DL_project/dataset/call_df_21.csv"
#test_="/content/gdrive/My Drive/DL_project/test/Test.csv"
_data_filepath = os.path.join(_data_root, "call_df_21.csv")


shutil.copy(train_,_data_root)
#shutil.copy(test_,_data_root)


'/tmp/tfx-datagy8to2qo/call_df_21.csv'

In [14]:
context = InteractiveContext()
examples = external_input(_data_root)
example_gen = CsvExampleGen(input=examples)
context.run(example_gen)

Instructions for updating:
external_input is deprecated, directly pass the uri to ExampleGen.


INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.


INFO:absl:Processing input csv data /tmp/tfx-datagy8to2qo/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/tmp/tfx-interactive-2020-12-07T01_48_11.028514-t7es0csu/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:2884897,xor_checksum:1607305691,sum_checksum:1607305691"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "examples"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "pipeline_name"
          value {
            string_value: "interactive-2020-12-07T01_48_11.028514"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "CsvExampleGen"
          }
        }
        custom_properties {
          key: "span"
          value {
            string_value: "0"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        ))

In [15]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] /tmp/tfx-interactive-2020-12-07T01_48_11.028514-t7es0csu/CsvExampleGen/examples/1


In [16]:
# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: ""
    value {
      int64_list {
        value: 1661
      }
    }
  }
  feature {
    key: "Ask"
    value {
      float_list {
        value: 22.0
      }
    }
  }
  feature {
    key: "Bid"
    value {
      float_list {
        value: 20.399999618530273
      }
    }
  }
  feature {
    key: "Strike"
    value {
      float_list {
        value: 40.0
      }
    }
  }
  feature {
    key: "UnderlyingPrice"
    value {
      float_list {
        value: 61.04999923706055
      }
    }
  }
  feature {
    key: "date_diff"
    value {
      float_list {
        value: 16.0
      }
    }
  }
  feature {
    key: "sigma_21"
    value {
      float_list {
        value: 0.029875371605157852
      }
    }
  }
  feature {
    key: "treasury_rate"
    value {
      float_list {
        value: 1.4299999475479126
      }
    }
  }
}

features {
  feature {
    key: ""
    value {
      int64_list {
        value: 1665
      }
    }
  }
  feature {
    key: "As

In [17]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:We decided to produce LargeList and LargeBinary types.
INFO:absl:We decided to produce LargeList and LargeBinary types.
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to /tmp/tfx-interactive-2020-12-07T01_48_11.028514-t7es0csu/StatisticsGen/statistics/2/train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to /tmp/tfx-interactive-2020-12-07T01_48_11.028514-t7es0csu/StatisticsGen/statistics/2/eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "/tmp/tfx-interactive-2020-12-07T01_48_11.028514-t7es0csu/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "pipeline_name"
          value {
            string_value: "interactive-2020-12-07T01_48_11.028514"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
        ))

#**converting call_df to tf.records**

In [16]:
call_df_21.dtypes

UnderlyingPrice    float64
Strike             float64
Bid                float64
Ask                float64
sigma_21           float64
date_diff          float64
treasury_rate      float64
dtype: object

In [17]:
call_df_21.shape

(49833, 7)

In [18]:
pip install tqdm

In [13]:
""" Example module to convert csv data to TFRecords
"""
import csv
import tensorflow as tf
from tqdm import tqdm

def _bytes_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[value.encode()])
    )

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
        

original_data_file = "/content/gdrive/MyDrive/DL_project/dataset/call_df_21.csv"
tfrecords_filename = "/content/gdrive/MyDrive/DL_project/dataset/call_df_21.tfrecords"
tf_record_writer = tf.io.TFRecordWriter(tfrecords_filename)

with open(original_data_file) as csv_file:
    reader = csv.DictReader(csv_file, delimiter=",", quotechar='"')
    for row in tqdm(reader):
        example = tf.train.Example(
            features=tf.train.Features(
                feature={
                    "UnderlyingPrice": _bytes_feature(row["UnderlyingPrice"]),
                    "Strike": _bytes_feature(row["Strike"]),
                    "Bid": _bytes_feature(row["Bid"]),
                    "Ask": _bytes_feature(row["Ask"]),
                    "sigma_21": _bytes_feature(row["sigma_21"]),
                    "date_diff": _bytes_feature(row["date_diff"]),
                    "treasury_rate": _bytes_feature(row["treasury_rate"]),                    
                }
            )
        )
        tf_record_writer.write(example.SerializeToString())
    tf_record_writer.close()

49833it [00:03, 12809.94it/s]


#Data Ingestion and Visualization

In [20]:
import tensorflow_data_validation as tfdv
stats = tfdv.generate_statistics_from_tfrecord(data_location='/content/gdrive/MyDrive/DL_project/dataset/call_df_21.tfrecords')
tfdv.visualize_statistics(stats)


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [21]:
schema = tfdv.infer_schema(stats)
tfdv.display_schema(schema)


,Type,Presence,Valency,Domain
Feature name,,,,
'Ask',BYTES,required,,-
'Bid',BYTES,required,,-
'Strike',STRING,required,,'Strike'
'UnderlyingPrice',BYTES,required,,-
'date_diff',BYTES,required,,-
'sigma_21',BYTES,required,,-
'treasury_rate',BYTES,required,,-


/usr/local/lib/python3.6/dist-packages/tensorflow_data_validation/utils/display_util.py:138: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'Strike',"'100.0', '105.0', '110.0', '115.0', '120.0', '125.0', '130.0', '135.0', '140.0', '145.0', '150.0', '155.0', '160.0', '165.0', '170.0', '175.0', '180.0', '185.0', '190.0', '195.0', '200.0', '210.0', '30.0', '35.0', '40.0', '45.0', '50.0', '55.0', '60.0', '65.0', '70.0', '75.0', '80.0', '85.0', '90.0', '95.0'"


In [22]:
# generate two files from the existing data file for demo proposes
!sed -e '1,30000d;' /content/gdrive/MyDrive/DL_project/dataset/call_df_21.csv > /content/gdrive/MyDrive/DL_project/dataset/dataset_1.csv
!sed -e '1h;30000,49833d;' /content/gdrive/MyDrive/DL_project/dataset/call_df_21.csv> /content/gdrive/MyDrive/DL_project/dataset/dataset_2.csv

In [23]:
train_stats = tfdv.generate_statistics_from_csv(
    data_location='/content/gdrive/MyDrive/DL_project/dataset/dataset_1.csv',
    delimiter=',')
val_stats = tfdv.generate_statistics_from_csv(
    data_location='/content/gdrive/MyDrive/DL_project/dataset/dataset_1.csv',
    delimiter=',')

tfdv.visualize_statistics(lhs_statistics=val_stats, rhs_statistics=train_stats,
                          lhs_name='VAL_DATASET', rhs_name='TRAIN_DATASET')

In [24]:
anomalies = tfdv.validate_statistics(statistics=val_stats, schema=schema)
tfdv.display_anomalies(anomalies)


/usr/local/lib/python3.6/dist-packages/tensorflow_data_validation/utils/display_util.py:173: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Anomaly short description,Anomaly long description
Feature name,,
'\'106.52\'',New column,New column (column in data but not in schema)
'Ask',Column dropped,Column is completely missing
'\'0.0466434048839187\'',New column,New column (column in data but not in schema)
'Bid',Column dropped,Column is completely missing
'UnderlyingPrice',Column dropped,Column is completely missing
'sigma_21',Column dropped,Column is completely missing
'\'21.0\'',New column,New column (column in data but not in schema)
'Strike',Column dropped,Column is completely missing
'\'2.36\'',New column,New column (column in data but not in schema)


In [25]:
context.show(statistics_gen.outputs['statistics'])


#**TFX serving**

In [18]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update


deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0   130k      0 --:--:-- --:--:-- --:--:--  130k
OK
Hit:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launc

In [19]:
!apt-get install tensorflow-model-server


Reading package lists... Done
Building dependency tree       
Reading state information... Done
tensorflow-model-server is already the newest version (2.3.0).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [20]:
import sys

# Confirm that we're using Python 3
assert sys.version_info.major is 3, 'Oops, not running Python 3. Use Runtime > Change runtime type'

In [21]:
# TensorFlow and tf.keras
print("Installing dependencies for Colab environment")
!pip install -Uq grpcio==1.26.0

import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess

print('TensorFlow version: {}'.format(tf.__version__))

Installing dependencies for Colab environment
ERROR: tfx 0.23.1 has requirement grpcio<2,>=1.28.1, but you'll have grpcio 1.26.0 which is incompatible.
ERROR: google-cloud-spanner 1.19.1 has requirement google-cloud-core<2.0dev,>=1.4.1, but you'll have google-cloud-core 1.0.3 which is incompatible.
ERROR: google-cloud-bigtable 1.6.1 has requirement google-cloud-core<2.0dev,>=1.4.1, but you'll have google-cloud-core 1.0.3 which is incompatible.
ERROR: apache-beam 2.25.0 has requirement grpcio<2,>=1.29.0, but you'll have grpcio 1.26.0 which is incompatible.
TensorFlow version: 2.3.0


In [22]:
from tensorflow.keras.models import load_model
# loading CNN model for facial expression recognition
model = load_model('/content/gdrive/MyDrive/DL_project/dataset/model3_call21_WIX_200.h5') 

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 400)               2400      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 400)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 400)               160400    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 400)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 400)               160400    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 400)               1

In [24]:
# Fetch the Keras session and save the model
# The signature definition is defined by the input and output tensors,
# and stored with the default serving key
import tempfile

MODEL_DIR = tempfile.gettempdir()
#version = 1
export_path = MODEL_DIR
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

print('\nSaved model:')
!ls -l {export_path}

export_path = /tmp

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /tmp/assets

Saved model:
total 13736
drwxr-xr-x 4 root root     4096 Dec  7 00:43 1
drwxr-xr-x 2 root root     4096 Dec  7 00:53 assets
srw------- 1 root root        0 Dec  6 21:37 drivefs_ipc.0
srw------- 1 root root        0 Dec  6 21:37 drivefs_ipc.0_shell
drwxr-xr-x 2 root root     4096 Dec  6 23:04 ngrok
-rw-r--r-- 1 root root 13773305 Dec  6 23:04 ngrok-stable-linux-amd64.zip
drwxr-xr-x 2 root root     4096 Dec  7 01:48 __pycache__
-rw-r--r-- 1 root root   143318 Dec  7 01:48 saved_model.pb
drwx------ 2 root root     4096 Dec  7 00:36 tfx-data4k972teh
drwx------ 2 root root     4096 Dec  7 00:42 tfx-datadko_i3ul
drwx------ 2 root root     4096 Dec  7 00:53 tfx-dataej8zneph
drwx------ 2 root root

In [25]:
!saved_model_cli show --dir {export_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['dense_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 5)
        name: serving_default_dense_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_4'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 2)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
W1207 01:48:44.592797 140193889802112 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/res

In [26]:
os.environ["MODEL_DIR"] = MODEL_DIR

In [27]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=first_model \
  --model_base_path="${MODEL_DIR}"

Starting job # 0 in a separate thread.


In [28]:
!tail server.log

To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2020-12-07 00:54:58.210420: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:199] Restoring SavedModel bundle.
2020-12-07 00:54:58.254873: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:183] Running initialization op on SavedModel bundle at path: /tmp/1
2020-12-07 00:54:58.262050: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:303] SavedModel load for tags { serve }; Status: success: OK. Took 73981 microseconds.
2020-12-07 00:54:58.263058: I tensorflow_serving/servables/tensorflow/saved_model_warmup_util.cc:59] No warmup data file found at /tmp/1/assets.extra/tf_serving_warmup_requests
2020-12-07 00:54:58.263216: I tensorflow_serving/core/loader_harness.cc:87] Successfully loaded servable version {name: first_model version: 1}
2020-12-07 00:54:58.264399: I tensorflow_serving/model_servers/server.cc:367] Running gRPC ModelServer at 0.0.0.0:8500 ...
[warn

In [29]:
call_X_train, call_X_test, call_y_train, call_y_test = train_test_split(call_df_21.drop(['Bid', 'Ask'], axis=1),
                                                                        call_df_21[['Bid', 'Ask']], test_size=0.1, random_state=42)

In [30]:
call_X_test.to_csv('/content/gdrive/MyDrive/DL_project/dataset/call_X_test.csv')

In [31]:
xs = np.array(call_X_test)


#**Model Predictions**

In [44]:
import json

data = json.dumps({"signature_name": "serving_default",
                   "instances": xs.tolist()})

In [45]:
data

'{"signature_name": "serving_default", "instances": [[141.96, 190.0, 0.040351047419522064, 225.0, 2.22], [82.85, 70.0, 0.02629471439129068, 37.0, 1.64], [97.4, 155.0, 0.03228772966546013, 571.0, 2.54], [110.11, 135.0, 0.06315234590095266, 42.0, 2.41], [109.1, 95.0, 0.016403341833589692, 140.0, 2.52], [137.46, 115.0, 0.026066182636784167, 4.0, 2.41], [79.65, 60.0, 0.026428436979249875, 228.0, 1.86], [105.2, 130.0, 0.02488153995771937, 178.0, 2.19], [79.35, 75.0, 0.02112959870762864, 318.0, 2.06], [97.4, 45.0, 0.03228772966546013, 116.0, 1.93], [111.96, 70.0, 0.04309717861046268, 17.0, 2.14], [106.3, 60.0, 0.03627265867160459, 15.0, 1.98], [96.6, 135.0, 0.06593577053311157, 63.0, 2.36], [90.65, 145.0, 0.05369590696567412, 744.0, 2.5], [108.15, 95.0, 0.01953874168830817, 130.0, 2.46], [101.34, 120.0, 0.05211642321528565, 93.0, 2.45], [89.4, 40.0, 0.02907330589461383, 93.0, 1.81], [141.96, 55.0, 0.040351047419522064, 225.0, 2.22], [125.75, 65.0, 0.0567472996325723, 696.0, 2.5], [122.65, 17

In [46]:
!pip install -q requests
import requests

headers = {"content-type": "application/json"}
json_response = requests.post(
    'http://localhost:8501/v1/models/first_model:predict',         
    data = data,
    headers = headers)

In [47]:
predictions = json.loads(json_response.text)['predictions']
output = predictions[0]

In [48]:
output

[4.62066, 4.80713463]

In [49]:
output

[4.62066, 4.80713463]

In [49]:
from keras.preprocessing import image
classes = ['Bid','Ask']
classes[np.argmax(output)]

'Ask'

#**TFX Model Deployment/Prediction on Flask**

In [32]:
!pip install flask-ngrok


In [33]:
pip install -U flask-cors

Requirement already up-to-date: flask-cors in /usr/local/lib/python3.6/dist-packages (3.0.9)


In [34]:
from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify, request


**Making POST request to TF server**

In [35]:
import json
import numpy as np
import requests
from keras.applications import inception_v3
from keras.preprocessing import image
headers = {"content-type": "application/json"}
payload = {"instances": xs.tolist()}
json_response = requests.post('http://localhost:8501/v1/models/first_model:predict', json = payload)
predictions = json.loads(json_response.text)['predictions']
output= predictions[0]
print(output)

[4.62066, 4.80713463]


In [40]:
app = Flask(__name__)
run_with_ngrok(app) 
#app.route('/hello/', methods=['GET', 'POST'])
#def hello_world():
#    return 'Hello, World!'


@app.route('/first_model/predict/', methods=['POST'])
def first_model():
    payload = {"instances": xs.tolist()
    }
    json_response = requests.post('http://localhost:8501/v1/models/first_model:predict', json= payload)
    predictions = json.loads(json_response.text)['predictions']
    output= predictions[0]
    return jsonify(output)
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://ac538b4b505a.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [07/Dec/2020 02:00:07] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Dec/2020 02:00:08] "GET /favicon.ico HTTP/1.1" 404 -
